In [134]:
# EDA 사전 세팅
%matplotlib inline

import os
from functools import reduce
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None) ## 모든 열을 출력한다.
import seaborn as sns  #데이터 시각화 
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
sns.set(font="AppleGothic", 

        rc={"axes.unicode_minus":False},

        style='darkgrid')

In [135]:
train = pd.read_csv("/Users/choejeehyuk/분석에 필요한 데이터/신용커드 예측 경진대회/df_train.csv")
test = pd.read_csv("/Users/choejeehyuk/분석에 필요한 데이터/신용커드 예측 경진대회/test_df.csv")
train.head(5)

,car,reality,child_num,income_type,family_type,house_type,occyp_type,credit,month_ap,family-child,family_size,income_total,DAYS_EMPLOYED,begin_month,new_age,aver_dep,year_income
0,N,Y,1,Commercial associate,Civil marriage,House / apartment,Laborers,1,3,2.0,3.0,247500.0,1540,5.0,31,82500.0,61875.0
1,Y,Y,0,Working,Married,House / apartment,Managers,2,3,2.0,2.0,450000.0,4434,22.0,52,225000.0,37500.0
2,N,Y,0,Commercial associate,Married,House / apartment,Sales staff,0,9,2.0,2.0,202500.0,2092,37.0,41,101250.0,40500.0
3,Y,Y,0,State servant,Married,House / apartment,Managers,2,10,2.0,2.0,157500.0,2105,26.0,41,78750.0,31500.0
4,N,Y,2,Working,Married,House / apartment,High skill tech staff,1,10,2.0,4.0,270000.0,4996,18.0,37,67500.0,20769.0


In [136]:
train = train.replace({'year_income' : np.inf}, 0)
#test = test.replace({'year_income' : np.inf}, 0) 

In [137]:
train['begin_month'] = round(abs(train['begin_month']))
#test['begin_month'] = round(abs(test['begin_month']))

In [138]:
def list_sep (data):
  cat_list = []
  int_list = []
  for i in data.columns:
    if data[i].dtypes == 'O':
      cat_list.append(i)
    else :
      int_list.append(i)

  return int_list, cat_list

In [139]:
int_list, cat_list = list_sep(train)
print('int : ',int_list)
print('cat : ',cat_list)

int :  ['child_num', 'credit', 'month_ap', 'family-child', 'family_size ', 'income_total', 'DAYS_EMPLOYED', 'begin_month', 'new_age', 'aver_dep', 'year_income']
cat :  ['car', 'reality', 'income_type', 'family_type', 'house_type', 'occyp_type']


In [141]:
for df in [train]:
    df['income_total'] = np.log1p(df['income_total'])
    df['aver_dep'] = np.log1p(df['aver_dep'])
    df['year_income'] = np.log1p(df['year_income'])

In [142]:
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from category_encoders.ordinal import OrdinalEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.cluster import KMeans

In [143]:
encoder = OrdinalEncoder(cat_list)
train[cat_list] = encoder.fit_transform(train[cat_list], train['credit'])
# f_test[cat_list] = encoder.transform(f_test[cat_list])

In [144]:
train

,car,reality,child_num,income_type,family_type,house_type,occyp_type,credit,month_ap,family-child,family_size,income_total,DAYS_EMPLOYED,begin_month,new_age,aver_dep,year_income
0,1,1,1,1,1,1,1,1,3,2.0,3.0,2.596684,1540,5.0,31,2.511270,2.487644
1,2,1,0,2,2,1,2,2,3,2.0,2.0,2.640271,4434,22.0,52,2.589556,2.445136
2,1,1,0,1,2,1,3,0,9,2.0,2.0,2.581617,2092,37.0,41,2.527755,2.451788
3,2,1,0,3,2,1,2,2,10,2.0,2.0,2.562422,2105,26.0,41,2.507487,2.429902
4,1,1,2,2,2,1,4,1,10,2.0,4.0,2.603147,4996,18.0,37,2.494849,2.392541
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18275,1,1,0,2,2,1,8,1,2,2.0,2.0,2.581617,803,44.0,35,2.527755,2.527755
18276,1,2,2,3,2,1,5,1,6,2.0,4.0,2.589556,1984,2.0,33,2.479691,2.460822
18277,2,2,0,2,1,2,5,2,7,2.0,2.0,2.609056,2015,25.0,28,2.556691,2.482971
18278,1,1,0,2,4,1,1,2,3,1.0,1.0,2.568744,107,59.0,28,2.568744,0.000000


In [145]:
kmeans_train = train.drop(['credit'], axis=1)
kmeans = KMeans(n_clusters=36, random_state=42).fit(kmeans_train)
train['cluster'] = kmeans.predict(kmeans_train)
#test['cluster'] = kmeans.predict(test)

In [146]:
train

,car,reality,child_num,income_type,family_type,house_type,occyp_type,credit,month_ap,family-child,family_size,income_total,DAYS_EMPLOYED,begin_month,new_age,aver_dep,year_income,cluster
0,1,1,1,1,1,1,1,1,3,2.0,3.0,2.596684,1540,5.0,31,2.511270,2.487644,18
1,2,1,0,2,2,1,2,2,3,2.0,2.0,2.640271,4434,22.0,52,2.589556,2.445136,27
2,1,1,0,1,2,1,3,0,9,2.0,2.0,2.581617,2092,37.0,41,2.527755,2.451788,31
3,2,1,0,3,2,1,2,2,10,2.0,2.0,2.562422,2105,26.0,41,2.507487,2.429902,31
4,1,1,2,2,2,1,4,1,10,2.0,4.0,2.603147,4996,18.0,37,2.494849,2.392541,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18275,1,1,0,2,2,1,8,1,2,2.0,2.0,2.581617,803,44.0,35,2.527755,2.527755,19
18276,1,2,2,3,2,1,5,1,6,2.0,4.0,2.589556,1984,2.0,33,2.479691,2.460822,31
18277,2,2,0,2,1,2,5,2,7,2.0,2.0,2.609056,2015,25.0,28,2.556691,2.482971,31
18278,1,1,0,2,4,1,1,2,3,1.0,1.0,2.568744,107,59.0,28,2.568744,0.000000,28


In [147]:
int_list = ['DAYS_EMPLOYED', 'begin_month', 'new_age']

# int_list_test = ['DAYS_EMPLOYED', 'begin_month', 'new_age']


scaler = MinMaxScaler()
train[int_list] = scaler.fit_transform(train[int_list])
#f_test[int_list_test] = scaler.transform(f_test[int_list_test])

In [148]:
train

,car,reality,child_num,income_type,family_type,house_type,occyp_type,credit,month_ap,family-child,family_size,income_total,DAYS_EMPLOYED,begin_month,new_age,aver_dep,year_income,cluster
0,1,1,1,1,1,1,1,1,3,2.0,3.0,2.596684,0.097031,0.083333,0.217391,2.511270,2.487644,18
1,2,1,0,2,2,1,2,2,3,2.0,2.0,2.640271,0.281409,0.366667,0.673913,2.589556,2.445136,27
2,1,1,0,1,2,1,3,0,9,2.0,2.0,2.581617,0.132199,0.616667,0.434783,2.527755,2.451788,31
3,2,1,0,3,2,1,2,2,10,2.0,2.0,2.562422,0.133028,0.433333,0.434783,2.507487,2.429902,31
4,1,1,2,2,2,1,4,1,10,2.0,4.0,2.603147,0.317215,0.300000,0.347826,2.494849,2.392541,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18275,1,1,0,2,2,1,8,1,2,2.0,2.0,2.581617,0.050076,0.733333,0.304348,2.527755,2.527755,19
18276,1,2,2,3,2,1,5,1,6,2.0,4.0,2.589556,0.125319,0.033333,0.260870,2.479691,2.460822,31
18277,2,2,0,2,1,2,5,2,7,2.0,2.0,2.609056,0.127294,0.416667,0.152174,2.556691,2.482971,31
18278,1,1,0,2,4,1,1,2,3,1.0,1.0,2.568744,0.005734,0.983333,0.152174,2.568744,0.000000,28
